In [ ]:
import nyisotoolkit
from nyisotoolkit import NYISOData, NYISOStat,NYISOVis
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error,r2_score
import matplotlib.pyplot as plt
from scipy.stats import skew
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from meteostat import Hourly, Point
from datetime import datetime
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

I used the NYISO API and got load and marginal cost data from the years 2022 and 2023. Then, I did the same using Meteostats' API. I found it to be easier to clean the data in R and then import back to Python. As I'm not sure how to use both languages together in the same notebook, I may release the work that I did in R separately.

In [ ]:
df = pd.read_csv("C:/Users/Khalil/Downloads/merged_data.csv")

#Change column type to datetime64[ns] for column: 'date_time'
df = df.astype({'date_time': 'datetime64[ns]'})



In [ ]:
#I'm assuming that a precipitation value of "NA" accounts for days where there was no precipitation.
df["prcp"] = df["prcp"].fillna(0)

#I will also assume that the wind directions that are missing are southbound winds from the north, especially
#since such rows always seem to have a wind speed value
df["wdir"] = df["wdir"].fillna(0)


#Drop the sun duration column because it's completely empty
df.drop(columns="tsun")

# Change column type to category for column: 'coco'
df = df.astype({'coco': 'category'})

# Rename column 'LBMP....MWHr..N.Y.C.' to 'Price(MWhr)'
df = df.rename(columns={'LBMP....MWHr..N.Y.C.': 'Price(MWhr)'})

# Rename column 'Marginal.Cost.Losses....MWHr..N.Y.C.' to 'Marginal_Cost_Losses(MWHr)'
df = df.rename(columns={'Marginal.Cost.Losses....MWHr..N.Y.C.': 'Marginal_Cost_Losses(MWHr)'})

# Rename column 'Marginal.Cost.Congestion....MWHr..N.Y.C.' to 'Marginal_Cost_Congestion(MWHr)'
df = df.rename(columns={'Marginal.Cost.Congestion....MWHr..N.Y.C.': 'Marginal_Cost_Congestion(MWHr)'})

# Rename column 'N.Y.C.' to 'Load'
df = df.rename(columns={'N.Y.C.': 'Load'})

For the weather condition code, less than 1% of the rows have missing values. The dictionary that the dataset
came with explains that some weather stations either don't report weather conditions unless they are significant
or they just don't report them at all. Ordinarily, I would use something like missForest for imputation or some other more sophisticated method but that's overkill for such a small proportion of missing data. It would be better to just use the mode of the column instead of just deleting them to retain as much data as possible.



In [ ]:
from sklearn.impute import SimpleImputer

# Impute with the most frequent value
imputer = SimpleImputer(strategy='most_frequent')
df['coco'] = imputer.fit_transform(df[['coco']])


I was a little concerned about the skewness present in the columns, so I made these code blocks to transform them using the Yeo-Johnson transformation. But I was also concerned with how doing this may affect interpretability afterwards. I ended up leaving the data as is when making the models but I left these code blocks here in case I ever change my mind.

In [ ]:
# Calculate skewness
print(df["LBMP....MWHr..N.Y.C."].skew())
print(df["Marginal_Cost_Congestion(MWHr)"].skew())
print(df["Marginal.Cost.Losses....MWHr..N.Y.C."].skew())
print(df["Load"].skew())
print(df["temp"].skew())
print(df["dwpt"].skew())
print(df["rhum"].skew())
print(df["prcp"].skew())
print(df["pres"].skew())
print(df["wdir"].skew())


#I tried to use sqrt, log, and BoxCox to transform the df but it either didn't work or yielded 
#unsatisfactory results. I'll use the Yeo-Johnson transformation instead to get a balance between sqrt and log
#while not needing all of the df to be positive like BoxCox

pt = PowerTransformer(method='yeo-johnson')

df["LBMP....MWHr..N.Y.C."] = pt.fit_transform(df[['LBMP....MWHr..N.Y.C.']])
df["Marginal_Cost_Congestion(MWHr)"] = pt.fit_transform(df[['Marginal_Cost_Congestion(MWHr)]])
df["Marginal_Cost_Losses(MWHr)"]= pt.fit_transform(df[['Marginal_Cost_Losses(MWHr)]])
df["Load"]= pt.fit_transform(df[['Load']])
df["temp"]= pt.fit_transform(df[['temp']])
df["dwpt"]= pt.fit_transform(df[['dwpt']])
df["rhum"]= pt.fit_transform(df[['rhum']])
df["prcp"]= pt.fit_transform(df[['prcp']])
df["pres"]= pt.fit_transform(df[['pres']])
df["wdir"]= pt.fit_transform(df[['wdir']])
df["coco"]= pt.fit_transform(df[['coco']])









In [ ]:
#Some of these columns are still skewed
print(df["LBMP....MWHr..N.Y.C."].skew())
print(df["Marginal_Cost_Congestion(MWHr)"].skew())
print(df["Marginal_Cost_Losses(MWHr)"].skew())
print(df["Load"].skew())
print(df["temp"].skew())
print(df["dwpt"].skew())
print(df["rhum"].skew())
print(df["prcp"].skew())
print(df["pres"].skew())
print(df["wdir"].skew())
print(df["coco"].skew())

df["Marginal_Cost_Congestion(MWHr)"] = np.sqrt(df['Marginal_Cost_Congestion(MWHr)'])
df["wdir"] = np.sqrt(df['wdir'])
df["prcp"] = np.sqrt(df['prcp'])

print("New skewness after sqrt transformation: \n")
print(df["Marginal_Cost_Congestion(MWHr)"].skew())
print(df['wdir'].skew())
print(df['prcp'].skew())

#Those three columns were very stubborn in terms of their skewness 
# but at the very least I reduced it from what it was orginally by a significant margin.


In [ ]:

# Extract components from the date_time column
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['hour'] = df['date_time'].dt.hour
df['minute'] = df['date_time'].dt.minute
df['second'] = df['date_time'].dt.second
df['day_of_week'] = df['date_time'].dt.dayofweek  # Monday=0, Sunday=6
df['day_of_year'] = df['date_time'].dt.dayofyear
df['week_of_year'] = df['date_time'].dt.isocalendar().week

# Display the DataFrame with new features
print(df.head())

Since we have two years worth of data, let's use one to train our model(s) and the other to test its performance.

In [ ]:
import pandas as pd


# Define the split date
split_date = '2023-01-01 05:00:00'


# Split the DataFrame into training and testing sets based on the split date
train_data = df[df['date_time'] < split_date]
test_data = df[df['date_time'] >= split_date]



In [ ]:


# Separate the features and response for training and testing sets
response_var = "Price(MWhr)"

X_train = train_data.drop(columns=[response_var,"date_time"])
y_train = train_data[response_var]
X_test = test_data.drop(columns=[response_var,"date_time"])
y_test = test_data[response_var]





print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))


Random Forest Creation/Validation

In [ ]:
# Random Forest
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)


# Evaluate
rf_rmse = root_mean_squared_error(y_test, rf_pred)
print(f'Random Forest RMSE: {rf_rmse}')

r_squared_forest = r2_score(y_test, rf_pred)
print(f'Random Forest R-squared: {r_squared_forest}')

# Plot predictions
plt.plot(y_test.index, y_test, label='Actual')
plt.plot(y_test.index, rf_pred, label='RF Predictions')
plt.legend()
plt.show() 